## 动态图转ONNX教程

#### 环境准备

In [ ]:
!pip install paddlepaddle paddle2onnx # required for paddle.onnx.export
!pip install onnx onnxruntime # optional for check and run ONNX model 

### 一、动态图组网

In [1]:
import paddle
from paddle import nn
from paddle.static import InputSpec


class LinearNet(nn.Layer):
    def __init__(self):
        super(LinearNet, self).__init__()
        self._linear = nn.Linear(784, 10)

    def forward(self, x):
        return self._linear(x)

### 二、调用paddle.onnx.export接口，导出模型为ONNX协议

In [2]:
# export to ONNX 
layer = LinearNet()
save_path = 'onnx.save/linear_net'
x_spec = InputSpec([None, 784], 'float32', 'x')
paddle.onnx.export(layer, save_path, input_spec=[x_spec])

2020-11-18 07:00:39 [INFO]	ONNX model saved in onnx.save/linear_net.onnx


### 三、验证模型的正确性

In [3]:
# check by ONNX
import onnx

onnx_file = save_path + '.onnx'
onnx_model = onnx.load(onnx_file)
onnx.checker.check_model(onnx_model)
print('The model is checked!')

The model is checked!


### 四、ONNXRuntime推理转换后的ONNX模型

In [4]:
import numpy as np
import onnxruntime

x = np.random.random((2, 784)).astype('float32')

# predict by ONNX Runtime
ort_sess = onnxruntime.InferenceSession(onnx_file)
ort_inputs = {ort_sess.get_inputs()[0].name: x}
ort_outs = ort_sess.run(None, ort_inputs)

print("Exported model has been predict by ONNXRuntime!")

Exported model has been predict by ONNXRuntime!


### 五、对比转换后模型和Paddle模型的推理结果

In [5]:
# predict by Paddle
layer.eval()
paddle_outs = layer(x)

# compare ONNX Runtime and Paddle results
np.testing.assert_allclose(
    ort_outs[0], paddle_outs.numpy(), rtol=1.0, atol=1e-05)

print("The difference of result between ONNXRuntime and Paddle looks good!")

The difference of result between ONNXRuntime and Paddle looks good!
